In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

In [2]:
# dir = Path("C:/Users/igorp/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
dir = Path("C:/Users/igornobrega/USF_Research/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
files = os.listdir(dir)

In [3]:
file_id = 2

print("File:", files[file_id])

lv = LV()
lv.from_pyvista_read(dir/files[file_id], identifier="elemTag", threshold=[0, 1])
lv.identify_surfaces(
  
  ab_ql=0.03, 
  ab_qh=0.70,
  
  alpha_atr=0.5,  # coeff for radial distance computation
  alpha_mtr=0.5,
  beta_atr=0.9,  # coeff for second radial distance computation
  beta_mtr=0.9,
  gamma_atr=89,
  gamma_mtr=89,
)

plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
# plotter.add_mesh(lvsurf.arrows, lighting=False, scalars="angles")
# plotter.add_points(kcenters, color="red", point_size=300)
plotter.add_mesh(lv._surface_mesh, 
                  scalars="LV_SURFS", 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=False,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()

File: Full_Heart_Mesh_11.vtk


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [4]:
lv.to_dict()

{'NODES': array([[ -3.87907733,  12.52942467, -19.55683257],
        [ -3.01636916,  13.5763117 , -19.82178605],
        [ -2.7924061 ,  12.583106  , -19.41994991],
        ...,
        [ 29.510462  , -38.928787  , -24.097198  ],
        [ 23.44714   , -25.526587  ,  15.682207  ],
        [ 21.786366  , -16.021004  , -45.794285  ]]),
 'ELEMENTS': {'TETRA': array([[    0,     1,     2,     3],
         [    4,     5,     6,     7],
         [    8,     9,    10,    11],
         ...,
         [72359, 72358, 50644, 72475],
         [50644, 50645, 72358, 50642],
         [50644, 72358, 50645, 72475]], dtype=int64)},
 'NODESETS': {'ENDO': array([    14,  74689,  51416, ..., 147819, 143164, 143163], dtype=int64),
  'AORTIC': array([    38,  27260,  44767, ..., 142515, 142131, 135242], dtype=int64),
  'MITRAL': array([    36, 119975,  31225, ..., 132048, 149154, 148776], dtype=int64),
  'AM_INTERCECTION': array([    36, 119975,  31225, ..., 142515, 142131, 148776], dtype=int64),
  'EPI': arr

In [5]:
lv.to_json("test.json")

In [4]:
from datetime import datetime
def runtime(start_time, end_time, ms=True):
    time_diff = (end_time - start_time)
    if ms:
        return time_diff.total_seconds() * 1000
    else:
        return time_diff.total_seconds()

In [5]:
del lv

In [6]:
runtimes = []

n = len(files)
for i in range(n):
  
  
  curr_file = dir/files[i]
  lv = LV()
  lv.from_pyvista_read(curr_file, identifier="elemTag", threshold=[0, 1])
  
  start_time = datetime.now()
  lv.identify_surfaces()
  end_time = datetime.now()
  
  
  exe_time = runtime(start_time, end_time, ms=False)
  runtimes.append(exe_time)
  print("{}: {:.6} [s]".format(files[i], exe_time))
  del lv
 
mean_exec_time = sum(runtimes)/n
print("Mean execution time: {:.6} [s]".format(mean_exec_time))

Full_Heart_Mesh_1.vtk: 2.34097 [s]
Full_Heart_Mesh_10.vtk: 2.01521 [s]
Full_Heart_Mesh_11.vtk: 2.06989 [s]
Full_Heart_Mesh_12.vtk: 2.46358 [s]
Full_Heart_Mesh_13.vtk: 2.2219 [s]
Full_Heart_Mesh_14.vtk: 2.10036 [s]
Full_Heart_Mesh_15.vtk: 2.25351 [s]
Full_Heart_Mesh_16.vtk: 1.80132 [s]
